**8.- Compara el modelo anterior con un modelo que use convoluciones 2D. Ten en cuenta que has de reordenar los datos para poder usar convoluciones 2D. Compara el resultado en unos datos de test con el resultado del modelo anterior. ¿Obtienes mejoras al utilizar las convoluciones 2D?**


In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Layer, Dense, GRU, Conv2D, Flatten, Dropout, BatchNormalization, MaxPooling2D, LSTM
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
#Leemos los datos de apple
apple = pd.read_csv(r"C:\Users\Carlos\Documents\MIA-X\Modulo 4\Machine Learning\Redes Convulcionales\CNN + RNN\practica_rnn_cnn\Stocks\aa.us.txt", delimiter=",")
amazon = pd.read_csv(r"C:\Users\Carlos\Documents\MIA-X\Modulo 4\Machine Learning\Redes Convulcionales\CNN + RNN\practica_rnn_cnn\Stocks\amzn.us.txt", delimiter=",")
microsoft = pd.read_csv(r"C:\Users\Carlos\Documents\MIA-X\Modulo 4\Machine Learning\Redes Convulcionales\CNN + RNN\practica_rnn_cnn\Stocks\msft.us.txt", delimiter=",")
facebook = pd.read_csv(r"C:\Users\Carlos\Documents\MIA-X\Modulo 4\Machine Learning\Redes Convulcionales\CNN + RNN\practica_rnn_cnn\Stocks\fb.us.txt", delimiter=",")
google = pd.read_csv(r"C:\Users\Carlos\Documents\MIA-X\Modulo 4\Machine Learning\Redes Convulcionales\CNN + RNN\practica_rnn_cnn\Stocks\googl.us.txt", delimiter=",")
#Me quedo con los datos de apertura

In [4]:
#Convertimos la columna Date como indice
for asset in [apple, amazon, microsoft, google, facebook]:
    asset.index = pd.to_datetime(asset.Date)

In [5]:
#Filtramos por el indice para homogeneizar los datos
data_close = apple[facebook.index[0]:].Close
apple = apple[facebook.index[0]:].Open
amazon = amazon[facebook.index[0]:].Open
microsoft = microsoft[facebook.index[0]:].Open
google = google[facebook.index[0]:].Open
facebook = facebook.Open

In [6]:
#Me genero una matriz de ceros con las dimensiones que quiero, n_ejemplos x 5 datos. 
LAG = 5
data = np.zeros((apple.shape[0]- LAG, LAG, 5))
close = np.zeros((data_close.shape[0]- LAG, LAG, 5))

In [7]:
#Unimos los datos en un único dataframe
for i in range(0,(apple.shape[0] - LAG)):
    data[i,:,0] = apple.iloc[i:i+LAG]
    data[i,:,1] = amazon.iloc[i:i+LAG]
    data[i,:,2] = microsoft.iloc[i:i+LAG]
    data[i,:,3] = google.iloc[i:i+LAG]
    data[i,:,4] = facebook.iloc[i:i+LAG]

In [8]:
x_train = data[:round(data.shape[0]*0.7),:-1,:]
x_test = data[round(data.shape[0]*0.7):,:-1,:]
y_train = close[:round(data.shape[0]*0.7),-1,0]
y_test = close[round(data.shape[0]*0.7):,-1,0]

Para poder utilizar capas convulcionales 2D es necesario reordenar nuestros datos para que tengan la dimensión del canal

In [11]:
#Reordenamos los datos para poder usar convulciones 2D
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

In [12]:
x_train.shape

(963, 4, 5, 1)

In [13]:
# Definimos un modelo con una capa convolucional 2D y 1 capa densa
model = Sequential()
model.add(Conv2D(50, 3, input_shape=(4,5,1), padding='same'))
model.add(Dropout(0.3))
model.add(MaxPooling2D(2))
model.add(Conv2D(30, 3, padding='same'))
model.add(Dropout(0.3))
model.add(MaxPooling2D(2))
model.add(Flatten())
model.add(Dense(1))

# mostramos el modelo
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 4, 5, 50)          500       
                                                                 
 dropout (Dropout)           (None, 4, 5, 50)          0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 50)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 2, 2, 30)          13530     
                                                                 
 dropout_1 (Dropout)         (None, 2, 2, 30)          0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 1, 30)         0         
 2D)                                                    

In [14]:
model.compile(optimizer='adam',loss='mae')

In [15]:
history = model.fit(x_train, y_train,  epochs=50)

Epoch 1/50
31/31 [==============================] - 0s 2ms/step - loss: 27.8849
Epoch 2/50
31/31 [==============================] - 0s 2ms/step - loss: 17.2842
Epoch 3/50
31/31 [==============================] - 0s 2ms/step - loss: 13.9536
Epoch 4/50
31/31 [==============================] - 0s 2ms/step - loss: 11.5457
Epoch 5/50
31/31 [==============================] - 0s 2ms/step - loss: 10.0842
Epoch 6/50
31/31 [==============================] - 0s 2ms/step - loss: 7.6657
Epoch 7/50
31/31 [==============================] - 0s 2ms/step - loss: 7.4142
Epoch 8/50
31/31 [==============================] - 0s 2ms/step - loss: 5.6603
Epoch 9/50
31/31 [==============================] - 0s 2ms/step - loss: 5.6493
Epoch 10/50
31/31 [==============================] - 0s 2ms/step - loss: 4.9127
Epoch 11/50
31/31 [==============================] - 0s 2ms/step - loss: 4.6523
Epoch 12/50
31/31 [==============================] - 0s 2ms/step - loss: 3.9431
Epoch 13/50
31/31 [=========================

In [16]:
model.evaluate(x_test, y_test)

13/13 [==============================] - 0s 1ms/step - loss: 0.5583


0.5583410263061523

Si lo comparamos con el resultado del ejercicio 7, vemos como el modelo anterior obtuvo un resultado bastante más favorable, con un erroor de 0.19. 